In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# importing dataset
dataset = pd.read_csv('train.csv')

In [3]:
dataset.isna().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [4]:
#filling nan values
dataset['education'].fillna(dataset['education'].mode()[0],inplace = True)
dataset['previous_year_rating'].fillna(dataset['previous_year_rating'].mode()[0],inplace = True)

In [5]:
dataset.drop('employee_id', axis=1, inplace=True)

In [6]:
X = dataset.drop('is_promoted', axis=1)
y = dataset['is_promoted']

In [7]:
X = pd.get_dummies(X, columns=['department', 'region', 'education', 'gender', 'recruitment_channel'], drop_first= True)

In [8]:
# splitting data into x_train,x_test, y_train, y_test
from sklearn.model_selection import train_test_split
x_train,x_test, y_train, y_test = train_test_split(X,y,test_size = 0.20, random_state = 100)

In [9]:
from datetime import datetime
class App_Logger:
    def __init__(self):
        pass

    def log(self, file_object, log_message):#This method will be used for logging all the information to the file.
        self.now = datetime.now()
        self.date = self.now.date()
        self.current_time = self.now.strftime("%H:%M:%S")
        file_object.write(str(self.date) + "/" + str(self.current_time) + "\t\t" + log_message +"\n")

In [10]:
rf_parameters = {'max_depth':[5,10,15,20,25,None],
                 'n_estimators':range(0,500,50),
                 'criterion':['gini','entropy'],
                 'bootstrap':[True,False],
                 'min_samples_split': range(2,10,1),
                 'max_features' : ['auto','log2'],
                 'min_samples_leaf':range(1,10),
                    }
xg_parameters = {"n_estimators"      : [10,50,100,200],
                "learning_rate"     : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
                 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15,20],
                 "min_child_weight" : [ 1, 3, 5, 7 ],
                 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4,0.5],
                 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    }
knn_parameters = {'n_neighbors':[50,100,200,250,300,350],
                 'weights':['uniform','distance'],
                 'algorithm':['ball_tree','kd_tree'],
                 'leaf_size':[20,25,30,35,40,45,50],
                     }
class ModelDevelopment:
    """
    This class shall be used for Building Optimized Model.  
    Written By: Akhil Sagar
    Version: 1.0
    Revisions: None
    
    """
    def __init__(self,file_object,logger_object):
        self.file_object = file_object
        self.logger_object = logger_object
        
    
    def get_randomforestclassifier_model(self, x_train, y_train):
        """
        Method Name: get_randomforestclassifier_model
        Description: This method will be used to build RandomForestClassifier model
        Input Description: It takes x_train and y_train data for training the model.

        Output:  It return Optimized RandomForestClassifier model.
        
        On Failure: Raise Exception

        Written By: Akhil Sagar 
        
        Version: 1.0
        
        Revisions: None

        """
        try:
            rmdsearch = RandomizedSearchCV(RandomForestClassifier(),rf_parameters,n_iter=10,cv=10,random_state=22, n_jobs=-1)
            rmdsearch.fit(x_train,y_train)
            hyperparameters =rmdsearch.best_params_
            max_depth,n_estimators,criterion,bootstrap,min_samples_split,max_features,min_samples_leaf=hyperparameters['max_depth'],hyperparameters['n_estimators'],hyperparameters['criterion'],hyperparameters['bootstrap'],hyperparameters['min_samples_split'],hyperparameters['max_features'],hyperparameters['min_samples_leaf']

            model = RandomForestClassifier(n_estimators=n_estimators,
                                           max_depth=max_depth,
                                           criterion=criterion,
                                           min_samples_leaf=min_samples_leaf,
                                           max_features=max_features,
                                           min_samples_split=min_samples_split,
                                           bootstrap=bootstrap,
                                           random_state=25,
                                           n_jobs=-1)
            model.fit(x_train, y_train)
            return model
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in performing Model Building and Tuning. Exception message: ' +str(e))
            raise Exception()
            
    def get_xgboost_model(self, x_train, y_train):
        """
        Method Name: get_xgboost_model
        Description: This method will be used to build XGBoost Classifier model
        Input Description: It takes x_train and y_train data for training the model.

        Output:  It return Optimized XGBoost model.
        
        On Failure: Raise Exception

        Written By: Akhil Sagar 
        
        Version: 1.0
        
        Revisions: None

        """
        try:
            rmdsearch = RandomizedSearchCV(XGBClassifier(objective='binary:logistic'), param_distributions = xg_parameters, n_iter=10,cv=10, n_jobs=-1)
            rmdsearch.fit(x_train,y_train)
            hyperparameters = rmdsearch.best_params_
            n_estimators,min_child_weight,max_depth,learning_rate,gamma,colsample_bytree = hyperparameters['n_estimators'],hyperparameters['min_child_weight'],hyperparameters['max_depth'],hyperparameters['learning_rate'],hyperparameters['gamma'],hyperparameters['colsample_bytree']
            xgboost_model = XGBClassifier(n_estimators=n_estimators,
                             learning_rate=learning_rate,
                             gamma=gamma,
                             min_child_weight=min_child_weight,
                             max_depth=max_depth,
                             colsample_bytree=colsample_bytree)
            xgboost_model.fit(x_train, y_train)
            return xgboost_model
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in performing Model Building and Tuning. Exception message: ' +str(e))
            raise Exception()
    def get_knnclassifier_model(self, x_train, y_train):
        """
        Method Name: get_knnclassifier_model
        Description: This method will be used to build KNearestNeighbour Classifier model
        Input Description: It takes x_train and y_train data for training the model.

        Output:  It return Optimized KNearestNeighbourClassifier model.
        
        On Failure: Raise Exception

        Written By: Akshay Anvekar
 
        
        Version: 1.0
        
        Revisions: None

        """
        try:
            rmdsearch = RandomizedSearchCV(KNeighborsClassifier(),knn_parameters,n_iter=10,cv=10,random_state=22, n_jobs=-1)
            rmdsearch.fit(x_train, y_train)
            hyperparameters =rmdsearch.best_params_
            n_neighbors,weights,algorithm,leaf_size = hyperparameters['n_neighbors'],hyperparameters['weights'],hyperparameters['algorithm'],hyperparameters['leaf_size']
            model = KNeighborsClassifier(n_neighbors=n_neighbors,
                                           weights=weights,
                                           algorithm=algorithm,
                                           leaf_size=leaf_size,
                                            n_jobs=-1)
            model.fit(x_train,y_train)
            return model
        except Exception as e:
            self.logger_object.log(self.file_object,'Exception occured in performing Model Building and Tuning. Exception message: ' +str(e))
            raise Exception()
logger_obj = App_Logger()
file = open("LogFile.txt", 'a+')
model=ModelDevelopment(file,logger_obj)

In [11]:
rf_model = model.get_randomforestclassifier_model(x_train,y_train)

In [12]:
xgboost_model = model.get_xgboost_model(x_train,y_train)

In [11]:
knn_model = model.get_knnclassifier_model(x_train, y_train)

In [13]:
rf_y_pred = rf_model.predict(x_test)
rf_y_pred

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [14]:
accuracy_score = accuracy_score(y_test,rf_y_pred)
accuracy_score

0.9355044699872286

In [15]:
xg_y_pred = xgboost_model.predict(x_test)

In [12]:
knn_y_pred = knn_model.predict(x_test)
knn_y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [16]:
knn_cross_val=cross_val_score(knn_model,x_train,y_train,cv=10,scoring='accuracy').mean()
knn_cross_val

0.923071764018011